In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report


In [ ]:
ticker = "AAPL"
stock_data = yf.download(ticker, start="2023-01-01", end="2023-12-31")
stock_data.reset_index(inplace=True)


In [ ]:
stock_data["Next_Close"] = stock_data["Close"].shift(-1)
stock_data["Target"] = (stock_data["Next_Close"] > stock_data["Close"]).astype(int)
stock_data.dropna(inplace=True)




In [ ]:
np.random.seed(42)

sentiment_data = pd.DataFrame({
    "Date": stock_data["Date"],
    "sentiment_score": np.random.uniform(-1, 1, len(stock_data)),
    "positive_score": np.random.uniform(0, 1, len(stock_data)),
    "negative_score": np.random.uniform(0, 1, len(stock_data))
})


In [ ]:
data = pd.merge(stock_data, sentiment_data, on="Date", how="inner")


In [ ]:
features = [
    "Open", "High", "Low", "Close", "Volume",
    "sentiment_score", "positive_score", "negative_score"
]

X = data[features]
y = data["Target"]

X.fillna(method="ffill", inplace=True)


In [ ]:
split = int(len(data) * 0.8)

X_train = X.iloc[:split]
X_test = X.iloc[split:]

y_train = y.iloc[:split]
y_test = y.iloc[split:]


In [ ]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
lr = LogisticRegression()
lr.fit(X_train_scaled, y_train)

y_pred_lr = lr.predict(X_test_scaled)

print("Logistic Regression")
print("Accuracy:", accuracy_score(y_test, y_pred_lr))
print("Precision:", precision_score(y_test, y_pred_lr))
print("Recall:", recall_score(y_test, y_pred_lr))
print("F1 Score:", f1_score(y_test, y_pred_lr))


In [ ]:
rf = RandomForestClassifier(n_estimators=200, max_depth=8, random_state=42)
rf.fit(X_train, y_train)

y_pred_rf = rf.predict(X_test)

print("Random Forest")
print("Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Precision:", precision_score(y_test, y_pred_rf))
print("Recall:", recall_score(y_test, y_pred_rf))
print("F1 Score:", f1_score(y_test, y_pred_rf))


In [ ]:
xgb = XGBClassifier(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.05,
    eval_metric="logloss",
    random_state=42
)

xgb.fit(X_train, y_train)

y_pred_xgb = xgb.predict(X_test)

print("XGBoost")
print("Accuracy:", accuracy_score(y_test, y_pred_xgb))
print("Precision:", precision_score(y_test, y_pred_xgb))
print("Recall:", recall_score(y_test, y_pred_xgb))
print("F1 Score:", f1_score(y_test, y_pred_xgb))


In [ ]:
pd.DataFrame({
    "Model": ["Logistic Regression", "Random Forest", "XGBoost"],
    "Accuracy": [
        accuracy_score(y_test, y_pred_lr),
        accuracy_score(y_test, y_pred_rf),
        accuracy_score(y_test, y_pred_xgb)
    ],
    "F1 Score": [
        f1_score(y_test, y_pred_lr),
        f1_score(y_test, y_pred_rf),
        f1_score(y_test, y_pred_xgb)
    ]
})
